In [2]:
import os
import sys
import cv2
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage.measure import compare_psnr
from skimage.measure import compare_ssim
MIN_MATCH_COUNT = 4
#特征点匹配 矫正图像
def features(img1,img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    ## (2) Create SIFT object
    sift = cv2.xfeatures2d.SIFT_create()

    ## (3) Create flann matcher
    matcher = cv2.FlannBasedMatcher(dict(algorithm = 1, trees = 5), {})

    ## (4) Detect keypoints and compute keypointer descriptors
    kpts1, descs1 = sift.detectAndCompute(gray1,None)
    kpts2, descs2 = sift.detectAndCompute(gray2,None)

    ## (5) knnMatch to get Top2
    matches = matcher.knnMatch(descs1, descs2, 2)
    # Sort by their distance.
    matches = sorted(matches, key = lambda x:x[0].distance)

        ## (6) Ratio test, to get good matches.
    good = [m1 for (m1, m2) in matches if m1.distance < 0.7 * m2.distance]

    canvas = img2.copy()

    ## (7) find homography matrix
    ## 当有足够的健壮匹配点对（至少4个）时
    if len(good)>MIN_MATCH_COUNT:
        ## 从匹配中提取出对应点对
        ## (queryIndex for the small object, trainIndex for the scene )
        src_pts = np.float32([ kpts1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kpts2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        ## find homography matrix in cv2.RANSAC using good match points
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        ## 掩模，用作绘制计算单应性矩阵时用到的点对
        #matchesMask2 = mask.ravel().tolist()
        ## 计算图1的畸变，也就是在图2中的对应的位置。
        h,w = img1.shape[:2]
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        ## 绘制边框
        cv2.polylines(canvas,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)
    else:
        print( "Not enough matches are found - {}/{}".format(len(good),MIN_MATCH_COUNT))


    ## (8) drawMatches
    matched = cv2.drawMatches(img1,kpts1,canvas,kpts2,good,None)#,**draw_params)

    ## (9) Crop the matched region from scene
    h,w = img1.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    perspectiveM = cv2.getPerspectiveTransform(np.float32(dst),pts)
    found = cv2.warpPerspective(img2,perspectiveM,(w,h),borderValue =[255,255,255])

    ## (10) save and display
    cv2.imwrite(folderName + "matched.png", matched)
    cv2.imwrite(folderName + "found.jpg", found)
    return found
    ##cv2.imshow("matched", matched);
    ##cv2.imshow("found", found);
    ##cv2.waitKey();cv2.destroyAllWindows()

#图像作差
def minusFirst(img1,img2):
    r,c=img1.shape[0],img1.shape[1]
    result=np.zeros((r,c,1))
    for i in range(r):
        for j in range(c):
            diff=img1[i,j]-img2[i,j]
            if diff==0:
                result[i,j]=0
            else:
                result[i,j]=255
    return result
#获取图像的结构相似度
def getPSNR(img1,img2):
    I2=cv2.resize(img2,(img1.shape[1],img1.shape[0]),interpolation=cv2.INTER_CUBIC)
    ##psnr=compare_psnr(img1,I2)
    (score,diff) = compare_ssim(img1,I2,full = True,multichannel=True)
    print(score)
    return score

#标注出差异的位置
def drawRect(src,img,psnr):
    global contours
    minArea=50##最小面积
    maxNum=1000#最大的轮廓个数
    #找轮廓
    image, contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    num=len(contours)
    ##print('轮廓个数：')
    ##print(num)
    #如果当前轮廓个数大于最大轮廓个数 进行前景像素腐蚀操作 并重新寻找轮廓
    while (num>maxNum):
        print('轮廓个数：')
        print(num)
        element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
        img=cv2.erode(img,element)
        cv2.imwrite(folderName + 'erode.jpg',img)
        img=cv2.imread(folderName + 'erode.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        retval, img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY) 
        image, contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        num=len(contours)
    if psnr>=0.9:#对于没有位移的图像 将膨胀的核设置更大一点 更精确显示图中的差异
        element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,4))
        img=cv2.dilate(img,element)
        cv2.imwrite(folderName + 'dilate.jpg',img)
        img=cv2.imread(folderName + 'dilate.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        retval, img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY) 
        image, contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    else:#对于有位移的图像 将膨胀的核设置小一些 避免将过多的干扰噪声标注出来
        element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
        img=cv2.dilate(img,element)
        cv2.imwrite(folderName + 'dilate.jpg',img)
        img=cv2.imread(folderName + 'dilate.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        retval, img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY) 
        image, contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#遍历轮廓 将大于最小面积的轮廓标注在原图上
    img.reshape()
    for i in range(len(contours)):
        x,y,w,h=cv2.boundingRect(contours[i])
        ##x,y,h,w=aRect
        ##print(aRect)
        tmpArea=w*h
       # print(tmpArea)
        if tmpArea<minArea:
            ##del(contours[i])
            continue
        cv2.rectangle(src,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
    return src



#并列显示合并两张图
def mergeImg(img1,img2):
    r=0
    if img1.shape[0]>img2.shape[0]:
        r=img1.shape[0]
    else:
        r=img2.shape[0]
    c=img1.shape[1]+img2.shape[1]
    result=np.zeros((r,c,img1.shape[2]))
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            result[i,j]=img1[i,j]
    for i in range(img2.shape[0]-1):
        for j in range(img2.shape[1]-1):
            result[i,img1.shape[1]+j]=img2[i,j]
    return result


start = time.clock()

global folderName,num
folderName = "E:\\tutorials\\images\\"
folderName = folderName+""

imgname1 =  folderName+"1.jpg"
imgname2 =  folderName+"2.jpg"

## (1) prepare data
img1 = cv2.imread(imgname1)
img2 = cv2.imread(imgname2)

#获取结构相似度
num = getPSNR(img1,img2)
if num==1:#两张图纸一样
    merge = mergeImg(img1,img2)
    cv2.imwrite(folderName+'merge.jpg',merge)
    print('两张图纸一样')
    sys.exit()
##print(num)
##先给两张图象的线条做一个膨胀
element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
##if num>0.9:##minus
    #gray
'''
    im_gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)   #转换了灰度化
    im_gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
     #线条膨胀 
    im_gray1 = cv2.erode(im_gray1,element)
    im_gray2 = cv2.erode(im_gray2,element)
    #binary
    retval, im_bin1 = cv2.threshold(im_gray1, 200, 255, cv2.THRESH_BINARY_INV) 
    retval, im_bin2 = cv2.threshold(im_gray2, 200, 255, cv2.THRESH_BINARY_INV) 
   
    minus=minusFirst(im_bin1,im_bin2);
else:
'''
img2 = features(img1,img2)
     #gray
im_gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)   #转换了灰度化
im_gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
     #线条膨胀 
im_gray1 = cv2.erode(im_gray1,element)
im_gray2 = cv2.erode(im_gray2,element)
    #binary
retval, im_bin1 = cv2.threshold(im_gray1, 200, 255, cv2.THRESH_BINARY_INV) 
retval, im_bin2 = cv2.threshold(im_gray2, 200, 255, cv2.THRESH_BINARY_INV) 
cv2.imwrite(folderName+"im_bin1.jpg",im_bin1);
cv2.imwrite(folderName+"im_bin2.jpg",im_bin2);
minus=minusFirst(im_bin1,im_bin2)

cv2.imwrite(folderName+'minus.jpg',minus)

#if num<0.9:
 #   num = getPSNR(img1,img2)
#close
global dilated
#要不要腐蚀还是要看连通域的个数 或者是轮廓的个数-
'''
if num<0.9:
    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    eroded = cv2.erode(minus,element)
    cv2.imwrite(folderName+'erode.jpg',eroded)
else:
'''
cv2.imwrite(folderName+'erode.jpg',minus)
##element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
##dilated = cv2.dilate(eroded,element)
##print(dilated.shape)

##cv2.imwrite('pics\\dilate.jpg',dilated)
result=cv2.imread(folderName+'erode.jpg')
result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
retval, result = cv2.threshold(result, 200, 255, cv2.THRESH_BINARY) 
#标注出差异
final = drawRect(img1,result,num)
cv2.imwrite(folderName+'final.jpg',final)
#并列合并两张图片
merge = mergeImg(final,img2)
cv2.imwrite(folderName+'merge.jpg',merge)
elapsed = (time.clock() - start)
print("Time used:",elapsed)


0.6107499420838117


F:\Anaconda3_5\lib\site-packages\ipykernel_launcher.py:81: RuntimeWarning: overflow encountered in ubyte_scalars


轮廓个数：
5474
轮廓个数：
1062


TypeError: reshape() takes exactly 1 argument (0 given)